Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.linear_algebra</a>
========

In [1]:
import proveit
from proveit import defaults
from proveit import a, b, c, X, Y, Z
from proveit.logic import InSet, InClass, CartExp
from proveit.numbers import Real, three
from proveit.linear_algebra import VecSpaces, VecAdd, ScalarMult
%begin demonstrations

### Vector space closure proofs under addition and scalar multiplication

In [2]:
R3 = CartExp(Real, three)

R3: Real^{3}

In [3]:
defaults.assumptions = [InSet(a, Real), InSet(b, Real), InSet(c, Real),
                        InSet(X, R3), InSet(Y, R3), InSet(Z, R3)]

defaults.assumptions: (a in Real, b in Real, c in Real, X in Real^{3}, Y in Real^{3}, Z in Real^{3})

In [4]:
InSet(ScalarMult(a, X), R3).prove()

{a in Real, X in Real^{3}} |- (a * X) in Real^{3}

In [5]:
InSet(VecAdd(X, Y, Z), R3).prove()

{X in Real^{3}, Y in Real^{3}, Z in Real^{3}} |- (X + Y + Z) in Real^{3}

In [6]:
lin_comb = VecAdd(ScalarMult(a, X), ScalarMult(b, Y), ScalarMult(c, Z))

lin_comb: (a * X) + (b * Y) + (c * Z)

In [7]:
InSet(lin_comb, R3).prove()

{a in Real, b in Real, c in Real, X in Real^{3}, Y in Real^{3}, Z in Real^{3}} |- ((a * X) + (b * Y) + (c * Z)) in Real^{3}

In [8]:
%end demonstrations